Задача 1 任务 1
Нужно написать две программы: Первая создает бинарный файл (min 2Гб), состоящий из случайных 32-разрядных беззнаковых целых чисел (big endian). Вторая считает сумму этих чисел (с применением длинной арифметики), находит минимальное и максимальное число.
您需要编写两个程序：第一个程序创建一个二进制文件（最小 2GB），由随机的 32 位无符号整数（大端序）组成。第二个计算这些数字的总和（使用长算术），找到最小和最大数字。

Реализуйте две версии -
实现两个版本 -

Простое последовательное чтение
简单的顺序读取
Многопоточная + memory-mapped files. Сравните время работы.
多线程 + 内存映射文件。比较开放时间。

创建二进制文件
Первая создает бинарный файл (min 2Гб), состоящий из случайных 32-разрядных беззнаковых целых чисел (big endian). 

In [1]:
import struct
import random

def create_file(filename, size=2):
    num_bytes = size * 1024*1024*1024 
    num_integers = num_bytes // 4  
    
    with open(filename, 'wb') as f:
        for _ in range(num_integers):
            integer = random.randint(0, 2**32-1)
            f.write(struct.pack('>I', integer))  

create_file('mydata')


读取文件并进行计算,
Вторая считает сумму этих чисел (с применением длинной арифметики), находит минимальное и максимальное число.

In [2]:
import os
import struct


def read_and_calculate(filename):
    sum_num = 0
    min_num = float('inf')
    max_num = float('-inf')

    with open(filename, 'rb') as f:
        while True:
            bytes_read = f.read(4)
            if not bytes_read:
                break
            num = struct.unpack('>I', bytes_read)[0]
            sum_num += num
            if num < min_num:
                min_num = num
            if num > max_num:
                max_num = num
    return sum_num, min_num, max_num

result = read_and_calculate('mydata')
print("Sum:", result[0], "Min:", result[1], "Max:", result[2])


Sum: 1152901223462466054 Min: 3 Max: 4294967294


多线程和内存映射文件.
Многопоточная + memory-mapped files

In [4]:
import mmap
import threading

def process_chunk(start, end, mm):
    sum_num = 0
    min_num = float('inf')
    max_num = float('-inf')
    
    for i in range(start, end, 4):
        num = struct.unpack('>I', mm[i:i+4])[0]
        sum_num += num
        if num < min_num:
            min_num = num
        if num > max_num:
            max_num = num
    return sum_num, min_num, max_num

def process_file(filename):
    with open(filename, 'rb') as file:
        mm = mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ)
        threads = []
        results = []
        size = len(mm) // 4  
        
        for i in range(4):
            start = i * size
            end = start + size
            thread = threading.Thread(target=lambda q, arg1, arg2, arg3: q.append(process_chunk(arg1, arg2, arg3)),args=(results, start, end, mm))
            threads.append(thread)
            thread.start()
        for thread in threads:
            thread.join()
        mm.close()

    summ= sum(x[0] for x in results)
    minn = min(x[1] for x in results)
    maxx = max(x[2] for x in results)
    return summ, minn, maxx

result =process_file('mydata')
print("Sum:", result[0], "Min:", result[1], "Max:", result[2])

Sum: 1152901223462466054 Min: 3 Max: 4294967294


比较两者时间.
 Сравните время работы.

In [5]:
import time

start_time = time.time()
read_and_calculate('mydata')
print("顺序读取时间 Простое последовательное чтение:{:.2f} s".format(time.time() - start_time))

start_time = time.time()
process_file('mydata')
print("多线程读取时间 Многопоточная + memory-mapped files:{:.2f} s".format(time.time() - start_time))


顺序读取时间 Простое последовательное чтение:120.08 s
多线程读取时间 Многопоточная + memory-mapped files:97.13 s
